<a href="https://colab.research.google.com/github/isold23/RAG/blob/main/search_from_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install llama-cpp-python langchain faiss-gpu sentence-transformers==2.2.2 InstructorEmbedding "unstructured[pdf]"

In [43]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (
    StreamingStdOutCallbackHandler
)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import sys

template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

print("sys.path[0] = ", sys.path[0])
print(os.listdir("/content/drive/MyDrive"))
os.chdir("/content/drive/MyDrive")
print("sys.path[0] = ", sys.path[0])


llm = LlamaCpp(
    model_path="/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q2_K.bin",
    #model_path="/drive/MyDrive/llama-2-13b-chat.ggmlv3.q4_0.bin",
    n_ctx=6000,
    n_gpu_layers=512,
    n_batch=30,
    callback_manager=callback_manager,
    temperature = 0.9,
    max_tokens = 4095,
    n_parts=1,

)

llm_chain = LLMChain(prompt=prompt, llm=llm)


#loader = UnstructuredFileLoader("AI discovers over 20K taxable French swimming pools.docx")
loader = UnstructuredFileLoader("/content/drive/MyDrive/lsmtree.pdf")

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# embedding engine
hf_embedding = HuggingFaceInstructEmbeddings()

db = FAISS.from_documents(docs, hf_embedding)

# save embeddings in local directory
db.save_local("faiss_AiArticle")

# load from local
db = FAISS.load_local("faiss_AiArticle/", embeddings=hf_embedding)

#query = "why french government using AI?"
query = "Introducing lsm tree"
search = db.similarity_search(query, k=2)
print("---------------------")
print(search)

template = '''Context: {context}

Based on Context provide me answer for following question
Question: {question}

Tell me the information about the fact. The answer should be from context only
do not use general knowledge to answer the query'''

prompt = PromptTemplate(input_variables=["context", "question"], template= template)
final_prompt = prompt.format(question=query, context=search)

llm_chain.run(final_prompt)



sys.path[0] =  /content
['llama-2-7b-chat.ggmlv3.q2_K.bin', 'lsmtree.pdf', 'Colab Notebooks', 'faiss_AiArticle', 'Wav2Lip', 'Google AI Studio']
sys.path[0] =  /content


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


max_seq_length  512
---------------------
[Document(page_content='5-\n\n2.1 How a Two Component LSM-tree Grows', metadata={'source': '/content/drive/MyDrive/lsmtree.pdf'}), Document(page_content='LSM-tree. Prior to this example, we offer an analytic derivation for the two component case.', metadata={'source': '/content/drive/MyDrive/lsmtree.pdf'})]


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 From the provided document, we can infer that LSM-tree is a two-component tree.

Explanation: The text "Introducing LSM-tree" appears in both documents, and it does not provide any information about the specific type of LSM-tree being discussed. This suggests that LSM-tree is a generic term that refers to some type of data structure or algorithm, rather than a specific implementation of a single tree.

In context, we can infer that LSM-tree is a two-component tree because the text in both documents mentions "two component" and "LSM-tree". However, it does not provide any information about the specific characteristics or properties of this tree.

' From the provided document, we can infer that LSM-tree is a two-component tree.\n\nExplanation: The text "Introducing LSM-tree" appears in both documents, and it does not provide any information about the specific type of LSM-tree being discussed. This suggests that LSM-tree is a generic term that refers to some type of data structure or algorithm, rather than a specific implementation of a single tree.\n\nIn context, we can infer that LSM-tree is a two-component tree because the text in both documents mentions "two component" and "LSM-tree". However, it does not provide any information about the specific characteristics or properties of this tree.'